In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()
import cv2
import matplotlib.pyplot as plt
import random
import multiprocessing
from keras.backend import set_session
from keras.backend import clear_session
from keras.backend import get_session
import time
from datetime import timedelta 


In [ ]:
# Disable gpu to use cpu
# tf.config.set_visible_devices([], 'GPU')

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    # tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
    # config = tf.compat.v1.ConfigProto()
    # config.gpu_options.per_process_gpu_memory_fraction = 0.6
    # keras.set_session(tf.Session(config=config))
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

In [ ]:
# Setup working directory
emotionList = ('Neutral', 'Happy', 'Sad', 'Surprise', 'Fear', 'Disgust', 'Anger', 'Contempt')
# limitCounter = [0,0,0,0,0,0,0,0]


def Setup():
  os.chdir("C:\\Users\\j.teoh\\Desktop\\tflite-facial-expression") # change working directory

# currently limited to 19999 images
# Load the images into a nested list containing info of image in tuple format (label, image name)
def LoadAllImageNames(imageFiles, limit = 0, catLimit = []*8):
  limitCounter = [0,0,0,0,0,0,0,0]
  trainingSet = []
  loadCounter = 0
  for file in imageFiles:
    if (limit > 0 and loadCounter > limit):
      break
  
    name = file.name[:-4] # file name w/o file extension
    data = np.load("train_set\\annotations\\{}_exp.npy".format(name))
    label = int(data.item(0))

    if limitCounter[label] >= catLimit[label]:
      continue
    limitCounter[label] += 1
    loadCounter += 1

    trainingSet.append((label, file.name))
    if (loadCounter%10000==0):
      print("Files loaded:{}".format(loadCounter))
  print("Total images loaded: ", loadCounter)
  print("Images Loaded: ", limitCounter)
  random.shuffle(trainingSet)
  return trainingSet

# Load the images into a nested list containing info of image in tuple format (label, image name)
def LoadAllTestImageNames(imageFiles):
  testingSet = []
  loadCounter = 0
  for file in imageFiles:
    loadCounter += 1
    if (loadCounter%10000==0):
      print("Files loaded:{}".format(loadCounter))
    name = file.name[:-4] # file name w/o file extension
    data = np.load("val_set\\annotations\\{}_exp.npy".format(name))
    label = int(data.item(0))
    if label != 1 and label != 2 and label != 6:
        continue
    testingSet.append((label, file.name))

  random.shuffle(testingSet)

  return testingSet

# Load the pixels of a picture to numpy.ndarray format. false for test set, true for training set
# Return image in RGB format
def LoadImage(imageName, trainingSetBool, normalize = True):
  intermediatePath = str()
  if (trainingSetBool):
    intermediatePath = "\\train_set\\images\\"
  else:
    intermediatePath = "\\val_set\\images\\"
  if normalize:
    image_array = cv2.imread("{}{}{}".format(os.getcwd(), intermediatePath, imageName))
    image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
    image_array = image_array/255
  else:
    image_array = cv2.imread("{}{}{}".format(os.getcwd(), intermediatePath, imageName))
    image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
  return image_array

# Extract the daata from 0 to amount from list and return it
def CropData(list, amount):
  if (len(list) < amount):
    amount = len(list)
  croppedList = list[:amount]
  del list[:amount]
  return croppedList

def LoadImages(list, trainingSetBool):
  label = []
  data = []
  count = 0
  for entries in list:
    try:
      image = LoadImage(entries[1], trainingSetBool)
      data.append(image) 
      label.append(entries[0]) 
    except: 
      print("Failed to load training image: ", entries[ 1])
  npLabel = np.array(label) 
  npData = np.array(data) 
  return npLabel, npData 

def InitializeModel():
  pretrained_model = tf.keras.applications.MobileNetV3Large(input_shape=(224,224,3)) # Initializing model with mobile net V3 pretrained model

  # Initializing the input and output from the model, removing last layer
  base_input = pretrained_model.layers[0].input
  base_output = pretrained_model.layers[-2].output

  # Adding 3 more layers to output side
  final_output = layers.Dense(128)(base_output) # Adding new layers, to the output side
  final_output = layers.Activation('relu')(final_output) # activating layer
  final_output = layers.Dense(64)(final_output)
  final_output = layers.Activation('relu')(final_output) # activating layer
  final_output = layers.Dense(8, activation = 'softmax')(final_output) # 8 cuz there are 8 image classifications

  new_model = keras.Model(inputs = base_input, outputs = final_output)
  return new_model

def ConvertToGray(image):
  image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
  return image

def ScaleImage(image, width):
  ratio = image.shape[1]/width
  image = cv2.resize(image, (width, int(image.shape[0]/ratio)))
  return image

def DetectFace(image):
  face_roi = np.ndarray(1)
  faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
  grayImage = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  faces = faceCascade.detectMultiScale(grayImage, 1.3, 5)
  for x,y,w,h in faces:
    roi_gray = grayImage[y:y+h, x:x+w]
    roi_color = image[y:y+h, x:x+w]
    cv2.rectangle(image, (x,y), (x+w, y+h), (255,0,0), 2)
    facess = faceCascade.detectMultiScale(roi_gray)
    if (len(facess) == 0):
      print("Face not detected")
    else:
      for (ex,ey,ew,eh) in facess:
        face_roi = roi_color[ey:ey+eh, ex:ex+ew]
  return face_roi

def ConvertToInput(image):
  input = ScaleImage(image, 224)
  input = np.expand_dims(input, axis = 0) ## to add fourth dimension to fit model input
  input = input/255
  return input

def GetResult(model, input):
  Predictions = model.predict(input)
  print(Predictions)
  result = np.argmax(Predictions)
  return emotionList[result]

# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    # try:
    #     del classifier # this is from global space - change this as you need
    # except:
    #     pass

    # use the same config as you used to create the session
    tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.9, visible_device_list="0")

class TimeHistory(keras.callbacks.Callback):
  def on_train_begin(self, logs={}):
    self.times = []

  def on_epoch_begin(self, batch, logs={}):
    self.epoch_time_start = time.time()

  def on_epoch_end(self, batch, logs={}):
    self.times.append(time.time() - self.epoch_time_start)

  def AverageTime(self):
    sum = 0
    for time in self.times:
      sum += time
    return sum/len(self.times)


In [ ]:
# Get currect directory (os.getcwd() -> C:\Users\jazzt\src)

#-----------------------Start of code---------------------------
# Path directories
Setup()

# initialise image names and label
imageFiles = os.scandir("train_set\\images")
trainingSetData = LoadAllImageNames(imageFiles, catLimit=[3000]*8)

In [ ]:
pretrained_model = tf.keras.applications.MobileNetV3Large(input_shape=(224,224,3)) # Initializing model with mobile net V3 pretrained model

# Initializing the input and output from the model, removing last layer
base_input = pretrained_model.layers[0].input
base_output = pretrained_model.layers[-2].output

# Adding 3 more layers to output side
final_output = layers.Dense(128)(base_output) # Adding new layers, to the output side
final_output = layers.Activation('relu')(final_output) # activating layer
final_output = layers.Dense(8, activation = 'softmax')(final_output) # 8 cuz there are 8 image classifications

model = keras.Model(inputs = base_input, outputs = final_output)

In [ ]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])

time_callback = TimeHistory()
dataSet = trainingSetData.copy()

# train model
batchSize = 48
imgPerIter = batchSize*16
count = 0
while(len(dataSet) != 0):
  # training data
  croppedList = []
  try:
    croppedList = CropData(dataSet, imgPerIter)
    print("loading image")
    label, data = LoadImages(croppedList, True)
    model.fit(data, label, epochs = 11, batch_size = batchSize, callbacks = [time_callback])
    print(time_callback.AverageTime())
  except RuntimeError as e:
    print(e)
  
  model.save_weights('bs48_128Dense.h5')
  count = count + len(croppedList)
  timeLeft = len(dataSet)/imgPerIter * time_callback.AverageTime()
  print("trained image count: ", count)
  print("Images Left: ", len(dataSet))
  print("Estimated completion time: ", str(timedelta(seconds=timeLeft)))
  reset_keras()


In [ ]:
# Default loading in model and training with reset of keras in GPU every iteration
# initialise model
model = InitializeModel()
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])

# train model

batchSize = 48
imgPerIter = batchSize*16
count = 0
while(len(trainingSetData) != 0):
  # training data
  try:
    croppedList = CropData(trainingSetData, imgPerIter)
    print("loading image")
    label, data = LoadImages(croppedList, True)
    model.fit(data, label, epochs = 11, batch_size = batchSize)
  except RuntimeError as e:
    print(e)
  
  model.save_weights('label126Limited6BatchSize48gpu.h5')
  count = count + imgPerIter
  print("trained image count: ", count)
  reset_keras()


In [ ]:
# load and save weights every iter, reset gpu memory with cuda (will break the program)


# train model

count = 0
while(len(trainingSetData) != 0):
  model = InitializeModel()
  model.load_weights("feModelWeights2feb.h5")
  model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])
  # training data
  try:
    croppedList = CropData(trainingSetData, 1000)
    print("loading image")
    label, data = LoadImages(croppedList, True)
    model.fit(data, label, epochs = 11, batch_size = 32)
  except RuntimeError as e:
    print(e)
  
  model.save_weights('feModelWeights2feb.h5')
  count = count + 1
  print("trained image count: ", count*1000)
  cuda.get_current_device().reset()


In [ ]:
#using multiprocessor to run model
# train model
count = 0
while(len(trainingSetData) != 0):

    croppedList = CropData(trainingSetData, 1000)
    print("loading image")
    p = multiprocessing.Process(target = trainModel, args=(croppedList))
    p.start()
    p.join()
    
  
    count = count + 1
    print("trained image count: ", count*1000)


In [ ]:
# Default loading and training of model, extracting lower half of data only

#-----------------------Start of code---------------------------
# Path directories
Setup()

# initialise image names and label
imageFiles = os.scandir("train_set\\images")
trainingSetData = LoadAllImageNames(imageFiles)

# initialise model
model = InitializeModel()
model.load_weights("feModelWeights5feb.h5")
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])
trainingSetData = trainingSetData[(120000+161000):]

# train model
count = 0
while(len(trainingSetData) != 0):
  # training data
  try:
    croppedList = CropData(trainingSetData, 1000)
    print("loading image")
    label, data = LoadImages(croppedList, True)
    model.fit(data, label, epochs = 11, batch_size = 32)
  except RuntimeError as e:
    print(e)
  
  model.save_weights('feModelWeights6feb.h5')
  count = count + 1
  print("trained image count: ", count*1000)


In [ ]:
# Count number of data in each facial expression
imageFiles = os.scandir("train_set\\images")
trainingSetData = LoadAllImageNames(imageFiles)

counterList = list(range(8))
for label, name in trainingSetData:
    counterList[label] += 1
    
print(counterList)
    

In [ ]:
sum = 0
for i in counterList:
    sum += i

for i in range(len(counterList)):
    counterList[i] /= sum

print(counterList)

In [ ]:
croppedList = CropData(trainingSetData, 1000)
label, data = LoadImages(croppedList, True)

In [ ]:
model = InitializeModel()
print(model.summary())
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])

In [ ]:
model = InitializeModel()
print(model.summary())
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])

In [ ]:
model.fit(data, label, epochs = 13, batch_size = 4)

In [ ]:
# Testing trained model
image = LoadImage("3.jpg", False, normalize = False)
# image = cv2.resize(image, (224,224))
image = ScaleImage(image, 224)

face = DetectFace(image)

plt.imshow(face)

preppedInput = ConvertToInput(face)
result = GetResult(model, preppedInput)


In [ ]:
result

In [ ]:
# testing loading in data from tar using tensorflow
os.chdir("C:\\Users\\j.teoh\\Desktop\\tflite-facial-expression") # change working directory
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, rotation_range=20)

In [ ]:
images, labels = next(img_gen.flow_from_directory("train_set.tar"))

In [ ]:
croppedList = CropData(trainingSetData, 1000)
label, data = LoadImages(croppedList, True)
dataset = tf.data.Dataset.from_tensor_slices((data,label)).batch(1)

In [ ]:
def show(image, label):
  plt.figure()
  plt.imshow(image)
  plt.title(label)
  plt.axis('off')

In [ ]:
model = InitializeModel()
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])


In [ ]:
model.fit(dataset.repeat(), epochs=11, steps_per_epoch=20)

In [ ]:
# Testing model
Setup()

modelNames = ["label126Limited6BatchSize48gpu.h5"]
model = InitializeModel()
mainData = LoadAllTestImageNames(os.scandir("val_set\\images"))
resultPool =[]

for modelName in modelNames:
    model.load_weights(modelName)
    model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])

    # initialise image names and label
    testSetData = []
    testSetData.extend(mainData)
    # print(mainData)
    # test model
    lostSum = 0
    accuracySum = 0
    count = 0
    while(len(testSetData) != 0):
      # training data
      # try:
      croppedList = CropData(testSetData, 100)
      print("loading image")
      label, data = LoadImages(croppedList, False)
      result = model.evaluate(data, label, batch_size = 1)
      lostSum += result[0]
      accuracySum += result[1]
      # except:
      #   print("Failed to train data")

      count += 1
      reset_keras()
      # print(count)
    # print(mainData)
    print("==========FINISH TESTING===========")
    print("model name: ", modelName)
    print("average lost: ", lostSum/count)
    print("average accuracy: ", accuracySum/count)
    resultPool.append((modelName, lostSum/count, accuracySum/count))
    
print(resultPool)

## Setting up image with Keras for MobileNet

In [ ]:
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input

Setup()
# preparing dataset of 1000 images
dataset = trainingSetData[:1000].copy()
trainingSet = []
for label, fileName in dataset:
    # Load image and convert to 224,224,3 nested array
    tempImg = image.load_img("train_set\\images\\"+fileName, target_size = (224,224))
    tempImg = image.img_to_array(tempImg)
    
    # turn all the values in nested array into value between -1 and 1
    trainingSet.append(preprocess_input(tempImg))
    




In [ ]:
trainingSet